In [7]:
import pandas as pd
import sys
import os
sys.path.append("/Users/marc/Desktop/PostDoc/Mutation/signature5/notebooks")
import tools

col_order = ['Chr', 'Pos', 'Ref', 'Alt', 'Sample', 'Tissue_CellType', 'Compartment',
             'Individual', 'Study', 'Context', 'GeneralType', 'Type']
col_meta = ["Study", "Sample", "Individual", "Age", "Compartment", "Status", "Tissue_CellType"]

Read data

In [9]:
dat = pd.read_csv("df_from_R.csv.gz")

<ipython-input-9-f06570a1f288>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dat = pd.read_csv("df_from_R.csv.gz")


In [10]:
dat = dat.rename(columns = {"donor": "Individual", "sample": "Sample", 
                            "pos":"Pos", "ref":"Ref", "alt":"Alt"})
dat["Individual"] = dat.Individual.str[0:7]
dat["Chr"] = "chr" + dat["chrom"].astype(str)
dat["Tissue_CellType"] = "Liver"
dat["Compartment"] = "WGS"
dat["Study"] = "Ng2021"

In [11]:
dat = tools.keep_snps(dat).reset_index(drop="True")
dat = dat[~dat["Chr"].isin(["chrX", "chrY"])]

In [12]:
cosmic = tools.COSMIC()
dat = tools.annotate_context(dat, "../fasta/hg19.fa")
dat = dat.assign(GeneralType=tools.annotate_general_type(dat))
dat['Type'] = cosmic.fold(dat['GeneralType'])

In [13]:
dat["chrom_id"] = dat["Chr"] + dat["Pos"].astype(str) + dat["Alt"]

In [14]:
sdat = dat[dat.Individual=="PD36714"]

In [15]:
trees = sdat.groupby(["id", "Sample"]).size().reset_index()
trees.columns = ["id", "Sample", "n_muts"]

In [11]:
pmeta2 = pd.read_excel("41586_2021_3974_MOESM3_ESMM/Supplementary Table 2.xlsx", skiprows=2)
coverages = pmeta2.set_index("Sample")["Coverage"].to_dict()
burdens = dat.groupby("Sample")["id"].size().to_dict()
seen = []
miss = []
for k,v in burdens.items():
    if k in coverages:
        seen.append(k)
    else:
        miss.append(k)

In [14]:
dat.groupby("chrom_id").size().sort_values()

chrom_id
chr10100001407T      1
chr369436449A        1
chr369429085G        1
chr36942771T         1
chr369425064T        1
                  ... 
chr7145784901G      58
chr944189966C       70
chr1428368928C      87
chr562231849A       95
chr3192734974T     105
Length: 1946591, dtype: int64

In [17]:
len(dat.groupby("Sample").size())

1585

In [62]:
dat[col_order].to_csv("Ng2021.csv.gz", compression="gzip", index=False)

Metafile 

In [2]:
pmeta = pd.read_excel("41586_2021_3974_MOESM3_ESMM/Supplementary Table 1.xlsx", skiprows=4)
pmeta.assign(Study="Ng2021", Compartment="WGS", Tissue_CellType="Liver")
pmeta = pmeta.rename(columns={"Patient ID": "Individual", "Age at \noperation (yrs)":"Age"})

In [3]:
pmeta.replace("NORMAL","Normal")
pmeta["Status"] = pmeta.Aetiology

In [16]:
stat = pmeta.set_index("Individual")["Status"].to_dict()
cell2indv = dat.set_index("Sample")["Individual"].to_dict()
ages = pmeta.set_index("Individual")["Age"].to_dict()

In [27]:
smoking = pmeta.set_index("Individual")["Smoking \nstatus"].replace("Current", "smoker").replace("Never", "non-smoker").replace("Ex", "ex-smoker").to_dict()

In [28]:
l = list(dat.Sample.unique())
meta = pd.DataFrame(l)
meta.columns = ["Sample"]
meta["Study"] = "Ng2021"
meta["Tissue_CellType"] = "Liver"
meta["Individual"] = meta.Sample.map(cell2indv)
meta["Age"] = meta["Individual"].map(ages)
meta["Status"] = meta["Individual"].map(stat)
meta["Compartment"] = "WGS"
meta["Smoking"] = meta["Individual"].map(smoking)

In [29]:
meta = meta.replace("NORMAL", "Normal")

In [31]:
meta[col_meta].to_csv("Ng2021.tab.gz",sep="\t", index=False)

Checks

In [96]:
dat = pd.read_csv("df_from_R.csv.gz")
mutsigs = pd.read_csv("mutsigs_branches.csv.gz")

<ipython-input-96-22042237b653>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dat = pd.read_csv("df_from_R.csv.gz")


In [97]:
mutsigs

,Signature 0,Signature P5,Signature P16,Signature N1,Signature P20,Signature P45,Signature P4,Signature N2,Signature N3,Signature N4,...,Signature P2,Signature P21,Signature N10,Signature N11,Signature N12,Signature P22,Signature P33,Signature P18,Signature N13,Signature N14
0,2.649716,164.708644,0.129126,1.123295,0.036743,3.028697,2.903770,74.531149,0.499709,0.001050,...,0.000000,0.000000,0.001050,0.000000,0.014697,0.000000,0.000000,0.064038,0.007349,0.000000
1,58.775928,513.113706,0.024146,214.819232,36.090380,20.308626,19.461588,150.422140,0.392129,0.033804,...,0.000000,0.000000,0.028009,0.000000,0.000966,0.000000,0.000000,0.000000,0.023180,0.007727
2,6.395586,49.347310,0.333931,5.197655,0.037241,10.838483,0.613241,5.503034,0.667862,0.000000,...,0.000000,0.000000,0.017379,0.007448,0.039724,0.000000,0.000000,0.000000,0.009931,0.002483
3,4.258586,8.319192,0.303030,18.159192,2.930101,0.694141,0.815354,37.347879,0.067879,0.033939,...,0.000000,0.000000,0.003232,0.001616,0.000000,0.000000,0.000000,0.002424,0.003232,0.019394
4,7.303316,187.273368,0.016726,5.258282,0.114444,79.451735,123.442419,300.675487,0.201598,0.001761,...,0.000000,0.000000,0.003521,0.004402,0.002641,0.000000,0.000000,0.000000,0.003521,0.014085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,8.452535,146.447004,0.930428,1.346465,0.036391,1.217621,3.279119,34.631395,18.032206,0.011802,...,0.000984,0.001967,0.001967,0.000000,0.039342,0.000000,0.000000,0.000000,0.000000,0.015737
1009,61.804864,200.445707,0.148053,158.066365,0.060147,20.814448,0.273899,235.553779,0.148979,0.000925,...,0.000000,0.000000,0.010179,0.000000,0.004627,0.000000,0.000000,0.003701,0.003701,0.000000
1010,19.547171,1041.344583,0.085558,78.159213,0.216747,80.889467,0.669254,18.112647,0.318466,0.000000,...,0.000000,0.004753,0.004753,0.000000,0.009506,0.007605,0.000000,0.000000,0.116929,0.017112
1011,33.636302,307.795121,0.283137,120.947844,0.153047,69.126172,0.262730,115.689833,0.518659,0.000000,...,0.000000,0.007652,0.000000,0.000850,0.002551,0.002551,0.000000,0.000000,0.015305,0.008503
